In [50]:
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import glob
import rasterio
from rasterio import plot
from rasterio.plot import show
from rasterio.warp import transform_bounds
import os
import re
from spectral_analysis_functions import *
# Using OceanDataLab reader 
#from sentinel2_l1c import *
import numpy as np
from S2_read import *
from s2_angs import *
bands=['B04','B08','B03','B02'];  
# imgPath=S2path+'GRANULE/L1C_T10SBG_A032114_20230430T191523/IMG_DATA/'
# imgPath=S2path+'GRANULE/L1C_T10SBF_A032071_20230427T190624/IMG_DATA/'
# Test 13/04/2024
# S2path='/home/acuevas/Bureau/S2_ALEXANDRA/27_04_2023/S2B_MSIL1C_20230427T185919_N0509_R013_T10SCF_20230427T223909.SAFE/'

granule_path = os.path.join(S2path, 'GRANULE')

# Liste des répertoires dans GRANULE
granule_dirs = os.listdir(granule_path)

# Vérifie si au moins un répertoire est présent
if granule_dirs:
    # Sélectionne le premier répertoire
    granule_dir = granule_dirs[0]

    # Chemin vers le dossier IMG_DATA
    imgPath = os.path.join(S2path, 'GRANULE', granule_dir, 'IMG_DATA')
    print(imgPath)
else:
    print("Aucun répertoire trouvé dans GRANULE")
    

# Liste des fichiers dans IMG_DATA
img_files = os.listdir(imgPath)

# Expressions régulières pour les noms de fichiers correspondant à chaque bande
band_patterns = {
    'band2': re.compile(r'.*B02\.jp2$'),
    'band3': re.compile(r'.*B03\.jp2$'),
    'band4': re.compile(r'.*B04\.jp2$'),
    'band8': re.compile(r'.*B08\.jp2$')
}

# Dictionnaire pour stocker les chemins d'accès aux fichiers de chaque bande
band_paths = {}

# Recherche des fichiers correspondant à chaque bande
for band, pattern in band_patterns.items():
    matching_files = [file for file in img_files if pattern.match(file)]
    if matching_files:
        # Sélectionne le premier fichier trouvé
        band_paths[band] = os.path.join(imgPath, matching_files[0])
    else:
        print(f"Aucun fichier trouvé pour la bande {band}")

# Ouverture des fichiers raster
band2 = rasterio.open(band_paths['band2'], driver='JP2OpenJPEG') if 'band2' in band_paths else None
band3 = rasterio.open(band_paths['band3'], driver='JP2OpenJPEG') if 'band3' in band_paths else None
band4 = rasterio.open(band_paths['band4'], driver='JP2OpenJPEG') if 'band4' in band_paths else None
band8 = rasterio.open(band_paths['band8'], driver='JP2OpenJPEG') if 'band8' in band_paths else None


NameError: name 'S2path' is not defined

In [51]:
import os
import re
import rasterio
from rasterio.warp import transform_bounds, transform
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt

bands = ['B04', 'B08', 'B03', 'B02']
img_files = os.listdir(imgPath)
# Expressions régulières pour les noms de fichiers correspondant à chaque bande
band_patterns = {
    'band2': re.compile(r'.*B02\.jp2$'),
    'band3': re.compile(r'.*B03\.jp2$'),
    'band4': re.compile(r'.*B04\.jp2$'),
    'band8': re.compile(r'.*B08\.jp2$')
}

# Dictionnaire pour stocker les chemins d'accès aux fichiers de chaque bande
band_paths = {}

# Recherche des fichiers correspondant à chaque bande
for band, pattern in band_patterns.items():
    matching_files = [file for file in img_files if pattern.match(file)]
    if matching_files:
        # Sélectionne le premier fichier trouvé
        band_paths[band] = os.path.join(imgPath, matching_files[0])
    else:
        print(f"Aucun fichier trouvé pour la bande {band}")

# Fonction pour lire et afficher les données des bandes
def read_and_print_band_data(band, band_name):
    data = band.read(1)
    print(f"{band_name} data:\n", data)
    print(f"{band_name} median value:", np.median(data.flatten()))

# Fonction pour transformer les coordonnées des coins
def get_geographic_bounds(band):
    bounds = band.bounds
    left, bottom, right, top = bounds.left, bounds.bottom, bounds.right, bounds.top
    return transform_bounds(band.crs, 'EPSG:4326', left, bottom, right, top)

# Ouverture des fichiers raster et traitement
try:
    with rasterio.open(band_paths['band2'], driver='JP2OpenJPEG') as band2, \
         rasterio.open(band_paths['band3'], driver='JP2OpenJPEG') as band3, \
         rasterio.open(band_paths['band4'], driver='JP2OpenJPEG') as band4, \
         rasterio.open(band_paths['band8'], driver='JP2OpenJPEG') as band8:

        # Lire les données des bandes
        B8 = band8.read(1)
        B4 = band4.read(1)
        B2 = band2.read(1)

        # Obtenir les limites géographiques pour la bande 8
        minlon, minlat, maxlon, maxlat = get_geographic_bounds(band8)
        xres, yres = band8.res
        extent = [minlon, maxlon, minlat, maxlat]
        
        # Bande 8
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.imshow(B2, vmin=1000, vmax=3000, extent=extent, cmap='viridis', origin='upper')
        ax.set_title('Band 2')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        ax.set_xlim(minlon, maxlon)
        ax.set_ylim(minlat, maxlat)

        # Bande 4
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.imshow(B4, vmin=1200, vmax=2000, extent=extent, cmap='viridis', origin='upper')
        ax.set_title('Band 4')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        ax.set_xlim(minlon, maxlon)
        ax.set_ylim(minlat, maxlat)
        plt.savefig('bande4.png')

        plt.show()
    
#         B8=band8.read(1)
#         B8=np.transpose(B8)
#         B8=np.rot90(B8,1)

#         B4=band4.read(1)
#         B4=np.transpose(B4)
#         B4=np.rot90(B4,1)
        
        
#         B2=band2.read(1)
#         B2=np.transpose(B2)
#         B2=np.rot90(B2,1)

#         fig, ax = plt.subplots(figsize=(12, 8))
#      #  Choisir Bande
#         im=ax.pcolormesh(B8,vmin=1000,vmax=1800) 
#         plt.colorbar(im)
#         ax.set_xlabel('Longitude')
#         ax.set_ylabel('Latitude')
#         ax.set_xlim(minlon, maxlon)
#         ax.set_ylim(minlat, maxlat)
        

        fig, ax = plt.subplots(figsize=(12, 4))
        # ax.hist(B2.flatten(), bins=50, color='green', alpha=0.7)
        ax.hist(B4.flatten(), bins=50, color='green', alpha=0.7)

        ax.set_title('Histogram of Band 4')

        plt.show()


except RasterioIOError as e:
    print(f"Unable to open one of the files: {e}")
print(extent)    # Fonction pour transformer les coordonnées des coins


NameError: name 'imgPath' is not defined

In [1]:
def get_geographic_bounds(band):
    bounds = band.bounds
    left, bottom, right, top = bounds.left, bounds.bottom, bounds.right, bounds.top
    return transform_bounds(band.crs, 'EPSG:4326', left, bottom, right, top)

with rasterio.open(band_paths['band2'], driver='JP2OpenJPEG') as band2, \
     rasterio.open(band_paths['band3'], driver='JP2OpenJPEG') as band3, \
     rasterio.open(band_paths['band4'], driver='JP2OpenJPEG') as band4, \
     rasterio.open(band_paths['band8'], driver='JP2OpenJPEG') as band8:

    B8=band8.read(1)
    B8=np.transpose(B8)
    B8=np.rot90(B8,1)

    B4=band4.read(1)
    B4=np.transpose(B4)
    B4=np.rot90(B4,1)
    
    B2=band2.read(1)
    B2=np.transpose(B2)
    B2=np.rot90(B2,1)
    
    # Obtenir les limites géographiques pour la bande 8
    minlon, minlat, maxlon, maxlat = get_geographic_bounds(band8)
    xres, yres = band8.res
    extent = [minlon, maxlon, minlat, maxlat]
    
#     # mother_box=[4000,10000,7500,10200]
#     mother_box=[4000,10000,6000,10200]

#     boxi1=mother_box
#     box_fille1=[7600,7800,8200,8400]
#     boxi=box_fille1
#     fig, ax = plt.subplots(figsize=(12, 8))
    
     #  Choisir Bande
    med_B2=np.nanmedian(B2)
    stdB2=np.nanstd(B2)
    im=ax.pcolormesh(B2,vmin=med_B2-2*stdB2,vmax=med_B2+2*stdB2) 
    plt.colorbar(im)


    plt.show()
print('minlon',minlon)
print('minlat',minlat)
print('maxlon',maxlon)
print('maxlat',maxlat)

NameError: name 'rasterio' is not defined